<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Data_Engineering_Essentials(news%20sentiment%20analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Fetch Data from an API](https://requests.readthedocs.io/en/latest/)**

In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
payload = {"dx_hcc": "dx", "dos_year": "v24_2022", "drf": "CNA", "search": "I50814", "start": 0}

In [ ]:
response = requests.post(url = 'https://www.hccreference.com/api/search',headers={"content-type":"application/json"},json =payload)

In [ ]:
response.text

# **News Sentiment Analysis**

https://newsapi.org/

https://signup.snowflake.com/

In [ ]:
!pip install snowflake-connector-python

In [ ]:
#data extraction & transformation
import json
import requests
import datetime
from datetime import date

# Your News API key
api_key = '{Put your API Key here}'

end_day = date.today()
start_day = today - datetime.timedelta(days=3)

def extract_news_api():
    url_extractor = f"https://newsapi.org/v2/everything?q='USA Election'&from={start_day}&to={end_day}&sortBy=popularity&apiKey={api_key}&language=en"
    print("The URL to extract the data is: ", url_extractor)
    response = requests.get(url_extractor)
    data = response.json()

    if data['status'] == 'ok':
        news_data = []
        for article in data['articles']:
            news_title = article['title']
            timestamp = article['publishedAt']
            url_source = article['url']
            partial_content = article['content'] or ""

            if len(partial_content) >= 200:
                partial_content = partial_content[:199]

            if '.' in partial_content:
                trimmed_part = partial_content[:partial_content.rindex('.')]
            else:
                trimmed_part = partial_content

            news_data.append((news_title, timestamp, url_source, trimmed_part))

        return news_data

In [ ]:
news_data=extract_news_api()

In [ ]:
news_data

In [ ]:
#loading data in Snowflake
import snowflake.connector

# Replace with your actual Snowflake credentials
SNOWFLAKE_ACCOUNT = ''
SNOWFLAKE_USER = ''
SNOWFLAKE_PASSWORD = ''
SNOWFLAKE_DATABASE = ''
SNOWFLAKE_SCHEMA = ''
SNOWFLAKE_TABLE = ''
SNOWFLAKE_WAREHOUSE = ''

def load_data_to_snowflake(data):
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )

    cursor = conn.cursor()

    # Assuming the table has columns: TITLE, TIMESTAMP, URL, CONTENT
    insert_query = f"""
    INSERT INTO {SNOWFLAKE_TABLE} (TITLE, TIMESTAMP, URL, CONTENT)
    VALUES (%s, %s, %s, %s)
    """

    try:
        cursor.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} rows inserted successfully.")
    except Exception as e:
        print("Error inserting data:", e)
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    news_data = extract_news_api()
    if news_data:
        load_data_to_snowflake(news_data)
